In [89]:
import numpy as np
import os
import pandas as pd
from sentence_transformers import SentenceTransformer, util

from sklearn.metrics.pairwise import cosine_similarity

import csv
import pickle
import time
import faiss

from sklearn.metrics.pairwise import cosine_similarity

import numpy as np
import os
import pandas as pd
# from sentence_transformers import SentenceTransformer, util
import re
# from sklearn.metrics.pairwise import cosine_similarity

import csv
import pickle
import time


#### 定义一些分句的函数

###### 这是给description用的，description比较脏，需要大量清洗

In [386]:
#helper code for description split
def cleaning_data(qc_data,app):
    app["description"] = app["description"].apply(lambda x : re.sub(r'(<)(.*?)(>)','',str(x)))#去除<.*>标签 
    app["description"] = app["description"].apply(lambda x : re.sub(r'(【)(.*?)(】)','',x))#去除<.*>标签 ，应该还可以加一些括号类！！《
    app["description"] = app["description"].apply(lambda x : re.sub(r'(《)(.*?)(》)','',x))
    app["description"] = app["description"].apply(lambda x : re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', x))#去除https：wwww.xxxx.com
    app["description"] = app["description"].apply(lambda x : re.sub(r'1\d{10}|(\d{3,4})-(\d{7,8})-\d{3,4}|\d{3,4}-\d{7,8}|\d{9}|\d{8}|\d{10}', '', x))#去除手机号类
    app["description"] = app["description"].apply(lambda x : re.sub(r'\w+([-+.]\w+)*@\w+([-.]\w+)*.\w+([-.]\w+)*.','', x))#去除邮箱
    app["description"] = app["description"].apply(lambda x : re.sub('|'.join(qc_data),'.',x))#去除特殊字符
    return app

def segmenting_sentences(fj_data,app):
    app1 = app.assign(description=app.description.str.split('['+''.join(fj_data)+']')).explode('description')#切分句子，切分后不会保留切分符号
    app1.reset_index()
    #有一些\n
    return app1

def remove_redundant_lines(dy_data,bh_data,app1):
    # 替换\n换行符及空格，返回非空格的行，即去除空格行：
    # dropna无法使用原因是因为处理后为中文字符串的空格并非nan
    app2 = app1.assign(description=app1.description.str.replace('\n','').replace('  ','').str.strip())#将换行符和空格替换为空
    app2= app2[app2["description"].str.contains('[\d\w]+')] # 匹配任意数字或字符
    app2 = app2[~app2['description'].isin(dy_data)] # 去掉description等于dy_data列表中元素的行
    app2 = app2[~app2["description"].str.contains('|'.join(bh_data))] # 去掉description包含dy_data列表中元素的行
    app2=app2[app2["description"].str.len()>5] # 长度小于5的不要了(因为有些分完很短感觉没用)
    return app2

In [387]:
#输入原始的一行description，或者abstract，叫data，这个data应该是一个一行一列的dataframe
##对于abstract和description各有一个最终的分句函数
def split_des(data): #分description
    app = data
    qc_data = ['&gt', '邮箱：', '■', '=-','💎','💖','🏆', '👧🏻','💄','🌟','✨','⚽','🔥','⭐','__________________________________________'
               ,'—','——','▲','◆','・','✔','=','=-','=---','=--','=+','=----','▶','📖','❂','▼','※','❤']
    app = cleaning_data(qc_data,app)
    fj_data = [';', '!', '?', '●', '•', '！', '!', '*', '✓', '【', '；', '✓', '★','。','.','✔️','=+','=-','=---','➤','®']  # 列表2：（还需补充,用来分割的符号）把想要变成句号的特殊放在这个列表中进行替换，
    app1 = segmenting_sentences(fj_data,app)
    dy_data = ['视觉革新', '品质升级','·&nbsp','Facebook:','Twitter:','Website:','updates','youtube']  # 列表3：若切分好句子后的description值为列表元素值，则删除该行，（delete whole sentence，必须精确匹配，unlike above）
    bh_data = ['取消订阅：', '客服热线','iPad','神马？','com/','」','iPhone','订阅','穿越火线','Instagram']  # 列表4：若切分好句子后的description值“”包含""列表元素值，则删除该行，（if contain this, delete whole sentence/row）
    app2 = remove_redundant_lines(dy_data,bh_data,app1)
    return app2   

###### 这是给abstract用的

In [388]:
#helper code for abstract split，这些html的还是需要去除掉的，因为有些abstract也有这些东西
def cleaning_abs(qc_data,app):
    app["abstract"] = app["abstract"].apply(lambda x : re.sub(r'(<)(.*?)(>)','',x))#去除<.*>标签 
    app["abstract"] = app["abstract"].apply(lambda x : re.sub(r'(【)(.*?)(】)','',x))#去除<.*>标签 ，应该还可以加一些括号类！！《
    app["abstract"] = app["abstract"].apply(lambda x : re.sub(r'(《)(.*?)(》)','',x))
    app["abstract"] = app["abstract"].apply(lambda x : re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', x))#去除https：wwww.xxxx.com
    app["abstract"] = app["abstract"].apply(lambda x : re.sub(r'1\d{10}|(\d{3,4})-(\d{7,8})-\d{3,4}|\d{3,4}-\d{7,8}|\d{9}|\d{8}|\d{10}', '', x))#去除手机号类
    app["abstract"] = app["abstract"].apply(lambda x : re.sub(r'\w+([-+.]\w+)*@\w+([-.]\w+)*.\w+([-.]\w+)*.','', x))#去除邮箱
    app["abstract"] = app["abstract"].apply(lambda x : re.sub('|'.join(qc_data),'.',x))#去除特殊字符
    return app

def segmenting_abs(fj_data,app):
    app1 = app.assign(abstract=app.abstract.str.split('['+''.join(fj_data)+']')).explode('abstract')#切分句子，切分后不会保留切分符号
    app1.reset_index()
    #有一些\n
    return app1


def remove_redundant_abs(dy_data,bh_data,app1):
    # 替换\n换行符及空格，返回非空格的行，即去除空格行：
    # dropna无法使用原因是因为处理后为中文字符串的空格并非nan
    app2 = app1.assign(abstract=app1.abstract.str.replace('\n','').replace('  ','').str.strip())#将换行符和空格替换为空
    app2= app2[app2["abstract"].str.contains('[\d\w]+')] # 匹配任意数字或字符
    app2 = app2[~app2['abstract'].isin(dy_data)] # 去掉description等于dy_data列表中元素的行
    app2 = app2[~app2["abstract"].str.contains('|'.join(bh_data))] # 去掉description包含dy_data列表中元素的行
    app2=app2[app2["abstract"].str.len()>5] # 长度小于5的不要了(因为有些分完很短感觉没用)
    return app2


In [389]:
# #abstrtact相对来说比较干净，可以适当去掉特殊字符以提高速度，此处dy_data，bh_data不能为空，
#不然报错，后面可以去abstarct里面找一些需要去除掉东西
def split_abstract(data):
    app = data
    qc_data = ['&gt']
    app = cleaning_abs(qc_data,app)
    fj_data = [';', '!', '?', '●', '•', '！', '!', '；','。','.']  # 列表2：（还需补充,用来分割的符号）把想要变成句号的特殊放在这个列表中进行替换，
    app1 = segmenting_abs(fj_data,app)
    dy_data = ['Twitter:','Website:','updates','youtube']  # 列表3：若切分好句子后的description值为列表元素值，则删除该行，（delete whole sentence，必须精确匹配，unlike above）
    bh_data = ['Instagram']  # 列表4：若切分好句子后的description值“”包含""列表元素值，则删除该行，（if contain this, delete whole sentence/row）
    app2 = remove_redundant_abs(dy_data,bh_data,app1)
    return app2
    

#### 导入 最终的 abstract和description的csv，此处各挑选11条做测试

In [77]:
#abstract data
final_abstract = pd.read_csv('./des50.csv',low_memory=False)

final_abstract = final_abstract.loc[:10] #只选取两列，11行


In [78]:
final_abstract.head()

,Unnamed: 0,Publication number,abstract
0,0,CA2966757A1,This disclosure provides a high quality protei...
1,1,CN104317959A,The invention claims an air blowing fan and el...
2,2,CN108345630A,The invention relates to a lithium battery cel...
3,3,CN108197330A,The invention relates to the field of detergen...
4,4,CN104317959B,The invention discloses a pressure cooker with...


In [79]:
final_abstract.shape

(11, 3)

In [80]:
#description data
final_app = pd.read_csv('abs50.csv',low_memory=False)
final_app = final_app[final_app['description'].notnull()].reset_index(drop=True)
final_app= final_app.loc[:10,['app_id','description']]#只选取两列，11行

In [81]:
final_app.head()

,app_id,description
0,284882215,"Connect with friends, family and people who sh..."
1,com.facebook.phone,Make your phone smarter with Hello. Built by M...
2,com.facebook.pages.app,"With Facebook Business Suite (Pages Manager), ..."
3,com.facebook.moments,Moments is going away soon and will be removed...
4,com.facebook.orca,"Be together whenever, with our free* all-in-on..."


In [82]:
final_app.shape

(11, 2)

In [106]:
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')#更换了model

In [84]:
from tqdm import tqdm #可以用这个查看运行进度
import time
top_k_hits =1 
pub_id = list()

simi = [] #空字典，用于存储

#开始循环
for i in tqdm(range(0,len(final_app))):
    for j in range(0,len(final_abstract)):
        description = final_app.loc[[i]] #对于每一个p
        Split_description = split_des(description).iloc[:,1].tolist() #对于每一个app的description拆句，转化list
        
        abstract = final_abstract.loc[[j]] #对于每一个patent
        Split_abstract = split_abstract(abstract).iloc[:,1].tolist() #对于每一个patent的abstract拆句，转化list
        
        Descrip_embed = model.encode(Split_description) ## 768 x description分出的句数 的大小 的矩阵
        Abstract_embed = model.encode(Split_abstract) ## 768 x abstract分出的句数 的大小 的矩阵
        
      
        all_simi = cosine_similarity(Descrip_embed,Abstract_embed)  #所有的similarity， 如拆10句description和
        #3句abstract，这里就是10x3大小的矩阵，里面每个都是similarity，一一对应
        
        #4 rules， 首先把矩阵变成一维的，进行排序
        top_1 = np.sort(all_simi.flatten())[-1:] # 最高的similarity 
        simi5 = np.mean(np.sort(all_simi.flatten())[-5:]) # top5/5
        simi10 = np.mean(np.sort(all_simi.flatten())[-10:]) # top10/10
        simi_all_avg = np.mean(all_simi.flatten()) # 所有similarity取平均
        
        simi.append({'app_id': final_app.iloc[i,0],'ab_id' : final_abstract.iloc[j,1], 'now_simi': all_simi,'simi5': simi5, 'simi10':simi10, 'simi_all_avg':simi_all_avg})
        #append 进去

100%|██████████| 11/11 [00:05<00:00,  1.97it/s]


In [85]:
# simi

##### 最终是这样子的dataframe，可以转化为csv方便日后调用，第一列是appid，第二列是abstract id， 后面几列是根据不同规则算出的，每一对app和patent 的similarity

In [87]:
last_data = pd.DataFrame(simi)
last_data

,app_id,ab_id,now_simi,simi5,simi10,simi_all_avg
0,284882215,CA2966757A1,"[[0.19491906, 0.19491906, 0.19491906], [0.1534...",0.192786,0.179527,0.128212
1,284882215,CN104317959A,"[[0.20607573], [0.15624644], [0.14147398], [0....",0.180460,0.160401,0.126515
2,284882215,CN108345630A,"[[0.19876027, 0.19876027, 0.19876027, 0.198760...",0.211383,0.206334,0.123458
3,284882215,CN108197330A,"[[0.19330287, 0.19330287, 0.19330287], [0.1499...",0.196981,0.177818,0.120995
4,284882215,CN104317959B,"[[0.185058], [0.1430866], [0.13890538], [0.110...",0.166798,0.148549,0.117373
...,...,...,...,...,...,...
116,com.facebook.groups,CN109558593A,"[[0.15921934, 0.15921934], [0.111110404, 0.111...",0.236298,0.211506,0.153906
117,com.facebook.groups,CN109165736A,"[[0.13611853, 0.13611853, 0.13611853, 0.136118...",0.225430,0.214328,0.139310
118,com.facebook.groups,CN108846636A,"[[0.16071177, 0.16071177, 0.16071177, 0.160711...",0.240663,0.234746,0.144638
119,com.facebook.groups,CN108875963A,"[[0.16699281, 0.16699281, 0.16699281, 0.166992...",0.245554,0.245029,0.152587


### 测试google maps

In [367]:
app = pd.read_csv("final_app_info2.csv")

In [368]:
# app = app.iloc[275:278]
final_app = app.loc[:,['name','app_id','unified_publisher_name','description']].reset_index(drop=True)

In [370]:
final_app.shape

(153930, 4)

In [401]:
final_app.head()
google = final_app.loc[final_app['unified_publisher_name'] == 'Google, Inc.'].dropna().reset_index(drop=True)
google.shape
google.head()

,name,app_id,unified_publisher_name,description
0,Facebook,284882215,"Facebook, Inc.","Connect with friends, family and people who sh..."
1,Hello — Caller ID & Blocking,com.facebook.phone,"Facebook, Inc.",Make your phone smarter with Hello. Built by M...
2,Facebook Business Suite (Pages Manager),com.facebook.pages.app,"Facebook, Inc.","With Facebook Business Suite (Pages Manager), ..."
3,Moments by Facebook,com.facebook.moments,"Facebook, Inc.",Moments is going away soon and will be removed...
4,Messenger – Text and Video Chat for Free,com.facebook.orca,"Facebook, Inc.","Be together whenever, with our free* all-in-on..."


(310, 4)

,name,app_id,unified_publisher_name,description
0,Wear OS by Google Smartwatch,com.google.android.wearable.app,"Google, Inc.","The Wear OS by Google app, previously Android ..."
1,Google Find My Device,com.google.android.apps.adm,"Google, Inc.",Find My Device helps you locate your lost Andr...
2,Voice Access,com.google.android.apps.accessibility.voiceaccess,"Google, Inc.",Voice Access helps anyone who has difficulty m...
3,Accessibility Scanner,com.google.android.apps.accessibility.auditor,"Google, Inc.",Accessibility Scanner is a tool that suggests ...
4,Google Wifi,com.google.android.apps.access.wifi.consumer,"Google, Inc.",The Google Wifi app will no longer be supporte...


In [454]:
abstract = pd.read_csv('mobile_navigation_AI_patent_for_google.csv')
abstract_all = pd.read_csv('final_abstract.csv')

In [456]:
# abstract  = abstract.iloc[25:26]
final_abstract = abstract.loc[:,['Publication number','abstract','CPC code (main)']].reset_index(drop=True)
final_abstract_all =abstract_all.loc[:,['Publication number','abstract']].reset_index(drop=True)
final_abstract.head()

,Publication number,abstract,CPC code (main)
0,AU2014241355A1,A system and method for facilitating clusterin...,G06Q30/0259
1,CA3092781A1,"Methods, systems, and apparatus for determinin...",G06N5/003
2,CN101203849A,Systems and methods for predictive conversion ...,G06F17/276
3,CN101467125B,"Methods, systems, and apparatus, including com...",G06F17/30684
4,CN105719001A,"Methods, systems, and apparatus, including com...",G06N3/08


In [393]:
final_abstract.loc[25:26]

,Publication number,abstract,CPC code (main)
25,DE202014010918U1,A system for enabling clustering of advertisem...,G06Q30/0259
26,DE202015009149U1,A non-transitory computer-readable medium havi...,G06F16/29


In [434]:
from tqdm import tqdm #可以用这个查看运行进度
import time
top_k_hits =1 
pub_id = list()


simi = [] #空字典，用于存储

#开始循环
for i in tqdm(range(0,len(google))):
    for j in range(25,26):
        description = google.loc[[i]] #对于每一个p
        Split_description = split_des(description).iloc[:,3].tolist() #对于每一个app的description拆句，转化list
        
        abstract = final_abstract.loc[[j]] #对于每一个patent
        Split_abstract = split_abstract(abstract).iloc[:,1].tolist() #对于每一个patent的abstract拆句，转化list
        
        Descrip_embed = model.encode(Split_description) ## 768 x description分出的句数 的大小 的矩阵
        Abstract_embed = model.encode(Split_abstract) ## 768 x abstract分出的句数 的大小 的矩阵
        
      
        all_simi = cosine_similarity(Descrip_embed,Abstract_embed)  #所有的similarity， 如拆10句description和
        #3句abstract，这里就是10x3大小的矩阵，里面每个都是similarity，一一对应
        
        #4 rules， 首先把矩阵变成一维的，进行排序
        top_1 = np.sort(all_simi.flatten())[-1:] # 最高的similarity 
        simi5 = np.mean(np.sort(all_simi.flatten())[-5:]) # top5/5
        simi10 = np.mean(np.sort(all_simi.flatten())[-10:]) # top10/10
        simi_all_avg = np.mean(all_simi.flatten()) # 所有similarity取平均
        
        simi.append({'app_id': google.iloc[i,0],'this_abstract' : final_abstract.iloc[j,1], 'now_simi': all_simi,'simi5': simi5, 'simi10':simi10, 'simi_all_avg':simi_all_avg})
        #append 进去

100%|██████████| 310/310 [10:51<00:00,  2.10s/it]


In [435]:
googlecompany = pd.DataFrame(simi)

In [436]:
googlecompany_5 = googlecompany.sort_values(by="simi5", ascending=False) 
googlecompany_5.to_csv('googlecompany_5.csv',encoding='utf-8-sig')

In [437]:
googlecompany_10 = googlecompany.sort_values(by="simi10", ascending=False) 
googlecompany_10.to_csv('googlecompany_10.csv',encoding='utf-8-sig')
googlecompany_all = googlecompany.sort_values(by="simi_all_avg" ,ascending=False) 
googlecompany_all.to_csv('googlecompany_all.csv',encoding='utf-8-sig')

In [438]:
googlecompany['simi5'].describe()
googlecompany['simi10'].describe()
googlecompany['simi_all_avg'].describe()

count    310.000000
mean       0.351660
std        0.075044
min        0.061733
25%        0.303422
50%        0.351648
75%        0.392052
max        0.539808
Name: simi5, dtype: float64

count    310.000000
mean       0.322164
std        0.071493
min        0.043033
25%        0.277613
50%        0.325091
75%        0.363909
max        0.512819
Name: simi10, dtype: float64

count    310.000000
mean       0.161809
std        0.040799
min        0.043033
25%        0.133815
50%        0.157755
75%        0.186975
max        0.307114
Name: simi_all_avg, dtype: float64

In [439]:
alibaba = final_app.loc[final_app['unified_publisher_name'] == 'Alibaba 阿里巴巴'].dropna().reset_index(drop=True)
alibaba.shape
alibaba

(181, 4)

,name,app_id,unified_publisher_name,description
0,停车大师3D 2 - 酷炫真实停车，极致驾驶体验,1100565890,Alibaba 阿里巴巴,最经典的3d停车游戏，全新续作——《停车大师3D 2》。\n 游戏基本继承了停车大师3d的...
1,采源宝-阿里巴巴专业微商/代购货源平台,1103629865,Alibaba 阿里巴巴,阿里巴巴专业微商/代购货源平台\nslogan：一手高利润源头厂货，积攒信任轻松赚钱\n\n...
2,Parking 3D - Free 3D Parking Game! Fun for All!,1107851872,Alibaba 阿里巴巴,It’s a 3D simulation game designed for those w...
3,UC News - Cricket & Timely news,1108364514,Alibaba 阿里巴巴,Highlights:\n\n Cricket Livescore -- Get t...
4,永恒纪元-暗影刺客,1137913384,Alibaba 阿里巴巴,—— 龙刃降世，再战纪元 ——\n“龙刃降世”东方神武·龙血利刃降世！炫酷技能特效，海量属性...
...,...,...,...,...
176,"UC Browser Turbo- Fast Download, Secure, Ad Block",com.ucturbo,Alibaba 阿里巴巴,<b>UC Browser Turbo 2020 is a new app of UC Br...
177,UC Browser Mini Hindi,com.uc.browser.hi,Alibaba 阿里巴巴,<div>UC Browser Mini in Hindi gives you a grea...
178,UC Browser HD for Tablet,com.uc.browser.hd,Alibaba 阿里巴巴,<div>\n<b>About UCWeb</b><br>Also check out UC...
179,"UC Video - Free Video Maker, Fast Video Downlo...",com.uc.browser.en.android.go,Alibaba 阿里巴巴,<b>UC Video</b> is a popular short video watch...


In [440]:
from tqdm import tqdm #可以用这个查看运行进度
import time
top_k_hits =1 
pub_id = list()

ali = [] #空字典，用于存储

#开始循环
for i in tqdm(range(0,len(alibaba))):
    for j in range(25,26):
        description = alibaba.loc[[i]] #对于每一个p
        Split_description = split_des(description).iloc[:,3].tolist() #对于每一个app的description拆句，转化list
        
        abstract = final_abstract.loc[[j]] #对于每一个patent
        Split_abstract = split_abstract(abstract).iloc[:,1].tolist() #对于每一个patent的abstract拆句，转化list
        
        Descrip_embed = model.encode(Split_description) ## 768 x description分出的句数 的大小 的矩阵
        Abstract_embed = model.encode(Split_abstract) ## 768 x abstract分出的句数 的大小 的矩阵
        
      
        all_simi = cosine_similarity(Descrip_embed,Abstract_embed)  #所有的similarity， 如拆10句description和
        #3句abstract，这里就是10x3大小的矩阵，里面每个都是similarity，一一对应
        
        #4 rules， 首先把矩阵变成一维的，进行排序
        top_1 = np.sort(all_simi.flatten())[-1:] # 最高的similarity 
        simi5 = np.mean(np.sort(all_simi.flatten())[-5:]) # top5/5
        simi10 = np.mean(np.sort(all_simi.flatten())[-10:]) # top10/10
        simi_all_avg = np.mean(all_simi.flatten()) # 所有similarity取平均
        
        ali.append({'app_id': alibaba.iloc[i,0],'this_abstract' : final_abstract.iloc[j,1], 'now_simi': all_simi,'simi5': simi5, 'simi10':simi10, 'simi_all_avg':simi_all_avg})
        #append 进去

100%|██████████| 181/181 [08:06<00:00,  2.69s/it]


In [441]:
ali

[{'app_id': '停车大师3D 2 - 酷炫真实停车，极致驾驶体验',
  'this_abstract': 'A system for enabling clustering of advertisements and map content, the system comprising:   one or more processors; and   a machine-readable medium comprising instructions stored therein which, when executed by the processors, cause the processors to perform operations comprising:   identifying an advertisement to be displayed to a user in response to a user request to the user device, wherein the user request is a search query for a destination region within the map;   determining a location associated with the advertisement;   determining a map region based on the determined location of the advertisement;   generating a query to retrieve one or more map content items, wherein the query includes one or more search criteria, and wherein the search criteria includes the determined map region;   obtaining one or more card content items having a location within the card region in response to the query, wherein the number of card

In [445]:
alicompany_5 = pd.DataFrame(ali).sort_values(by="simi5", ascending=False) 
alicompany_10 = pd.DataFrame(ali).sort_values(by="simi10",ascending=False) 
alicompany_all = pd.DataFrame(ali).sort_values(by="simi_all_avg", ascending=False) 
alicompany_5
alicompany_10
alicompany_all

,app_id,this_abstract,now_simi,simi5,simi10,simi_all_avg
179,"UC Video - Free Video Maker, Fast Video Downlo...",A system for enabling clustering of advertisem...,"[[0.25186083, 0.28351122, 0.17825252, 0.103141...",0.524940,0.489931,0.163977
162,高德地图,A system for enabling clustering of advertisem...,"[[0.3521345, 0.3098812, 0.20859046, 0.3415642,...",0.516855,0.484325,0.284236
167,UC Mini-Download Video Status & Movies,A system for enabling clustering of advertisem...,"[[0.318074, 0.2685945, 0.15725334, 0.13004205,...",0.516301,0.473670,0.153347
157,高德地图HD,A system for enabling clustering of advertisem...,"[[0.11018901, 0.21149229, 0.18867235, 0.132931...",0.508790,0.471324,0.245454
170,"UC Browser-Secure, Free & Fast Video Downloader",A system for enabling clustering of advertisem...,"[[0.2842027, 0.27462593, 0.14557755, 0.0897236...",0.504538,0.457880,0.154204
...,...,...,...,...,...,...
134,Youku,A system for enabling clustering of advertisem...,"[[0.20063782, 0.12509814, 0.13006519, 0.136030...",0.195672,0.166899,0.099424
96,淘宝大学-专业权威的电商培训平台,A system for enabling clustering of advertisem...,"[[0.17609946, 0.19855478, 0.1750196, 0.1346504...",0.195355,0.164667,0.126772
148,优酷XL高清影视,A system for enabling clustering of advertisem...,"[[0.25038025, 0.23963296, 0.12714064, 0.104883...",0.187720,0.145054,0.145054
91,师生家校,A system for enabling clustering of advertisem...,"[[0.03643861, 0.09607242, 0.10179724, 0.036555...",0.172706,0.108250,0.060631


,app_id,this_abstract,now_simi,simi5,simi10,simi_all_avg
179,"UC Video - Free Video Maker, Fast Video Downlo...",A system for enabling clustering of advertisem...,"[[0.25186083, 0.28351122, 0.17825252, 0.103141...",0.524940,0.489931,0.163977
162,高德地图,A system for enabling clustering of advertisem...,"[[0.3521345, 0.3098812, 0.20859046, 0.3415642,...",0.516855,0.484325,0.284236
167,UC Mini-Download Video Status & Movies,A system for enabling clustering of advertisem...,"[[0.318074, 0.2685945, 0.15725334, 0.13004205,...",0.516301,0.473670,0.153347
157,高德地图HD,A system for enabling clustering of advertisem...,"[[0.11018901, 0.21149229, 0.18867235, 0.132931...",0.508790,0.471324,0.245454
107,淘宝券券-附近优惠、团购、美食、ktv、自助餐、电影、摄影、吃喝玩乐、电子券、汽车服务,A system for enabling clustering of advertisem...,"[[0.3586241, 0.3524161, 0.3494018, 0.29921338,...",0.484217,0.460194,0.272725
...,...,...,...,...,...,...
96,淘宝大学-专业权威的电商培训平台,A system for enabling clustering of advertisem...,"[[0.17609946, 0.19855478, 0.1750196, 0.1346504...",0.195355,0.164667,0.126772
178,UC Browser HD for Tablet,A system for enabling clustering of advertisem...,"[[0.30766505, 0.2917251, 0.14831282, 0.1401119...",0.204285,0.159946,0.101105
148,优酷XL高清影视,A system for enabling clustering of advertisem...,"[[0.25038025, 0.23963296, 0.12714064, 0.104883...",0.187720,0.145054,0.145054
65,虾米音乐时代,A system for enabling clustering of advertisem...,"[[0.1311319, 0.09931813, 0.08246662, 0.0614217...",0.164573,0.143663,0.114067


,app_id,this_abstract,now_simi,simi5,simi10,simi_all_avg
162,高德地图,A system for enabling clustering of advertisem...,"[[0.3521345, 0.3098812, 0.20859046, 0.3415642,...",0.516855,0.484325,0.284236
93,爱淘宝—淘宝旗下的官方购物分享平台,A system for enabling clustering of advertisem...,"[[0.40740013, 0.3052562, 0.41896665, 0.3548458...",0.378155,0.342017,0.275626
107,淘宝券券-附近优惠、团购、美食、ktv、自助餐、电影、摄影、吃喝玩乐、电子券、汽车服务,A system for enabling clustering of advertisem...,"[[0.3586241, 0.3524161, 0.3494018, 0.29921338,...",0.484217,0.460194,0.272725
54,淘寶臺灣賣家版,A system for enabling clustering of advertisem...,"[[0.4561563, 0.46326625, 0.4031294, 0.34078613...",0.428915,0.400995,0.269701
123,淘寶Lite – 官方專為國際及港澳台用戶打造,A system for enabling clustering of advertisem...,"[[0.24928261, 0.20327607, 0.2813212, 0.2074195...",0.382509,0.366827,0.266061
...,...,...,...,...,...,...
153,优酷-这！就是街舞4 全网独播,A system for enabling clustering of advertisem...,"[[0.068201564, 0.01472242, 0.052803054, 0.0321...",0.372760,0.345435,0.096652
26,"大天使之剑H5-战宠降临,第六角色开启",A system for enabling clustering of advertisem...,"[[0.1366027, 0.08720502, 0.10354776, 0.0619219...",0.253864,0.231998,0.091598
39,末日沙城-大哥玩的传奇,A system for enabling clustering of advertisem...,"[[0.08267578, 0.10865853, 0.1057179, 0.0600569...",0.195703,0.187072,0.089805
137,Legend of Nine Tails,A system for enabling clustering of advertisem...,"[[0.084922194, 0.15182588, 0.17073657, 0.11581...",0.277747,0.259740,0.078619


In [446]:
alicompany_5.to_csv('alicompany_5.csv',encoding='utf-8-sig')
alicompany_10.to_csv('alicompany_10.csv',encoding='utf-8-sig')
alicompany_all.to_csv('alicompany_all.csv',encoding='utf-8-sig')

In [443]:
alicompany = pd.DataFrame(ali)
alicompany.shape
alicompany['simi5'].describe()
alicompany['simi10'].describe()
alicompany['simi_all_avg'].describe()

(181, 6)

count    181.000000
mean       0.355807
std        0.072821
min        0.164573
25%        0.310427
50%        0.369824
75%        0.398585
max        0.524940
Name: simi5, dtype: float64

count    181.000000
mean       0.327512
std        0.072450
min        0.108250
25%        0.287172
50%        0.338260
75%        0.373651
max        0.489931
Name: simi10, dtype: float64

count    181.000000
mean       0.174137
std        0.045193
min        0.060631
25%        0.145074
50%        0.171207
75%        0.202735
max        0.284236
Name: simi_all_avg, dtype: float64

In [470]:
final_abstract_all.loc[68466:68466]

,Publication number,abstract
68466,US11194973B1,"<abstract lang=""en"" load-source=""national-offi..."


In [463]:
amazon = final_app.loc[final_app['unified_publisher_name'] == 'AMZN Mobile LLC'].dropna().reset_index(drop=True)
amazon.shape
amazon

(136, 4)

,name,app_id,unified_publisher_name,description
0,AmazonDistribution,in.amazonbusiness.berrio.android.app,AMZN Mobile LLC,Now buy thousands of products from hundreds of...
1,"6PM - Shoes, Clothes & More",com.zappos.android.sixpmFlavor,AMZN Mobile LLC,The 6pm.com App puts the top brands and deals ...
2,"Zappos: Shoes, clothes, boots, coats, & more!",com.zappos.android,AMZN Mobile LLC,Zappos makes shopping easier! Shop all the top...
3,Whole Foods Market,com.wholefoods.wholefoodsmarket,AMZN Mobile LLC,The grocery store sidekick you’ve always wante...
4,Souq.com Sellers,com.souq.seller,AMZN Mobile LLC,Souq.com Sellers for Android just launched! Se...
...,...,...,...,...
131,Amazon ショッピングアプリ,374254473,AMZN Mobile LLC,Amazonの公式無料アプリ。本、日用品、ファッション、食品、ベビー用品ほか数億種類の商品を...
132,Amazon FR,358861688,AMZN Mobile LLC,"Complète\nRecherchez et achetez des produits, ..."
133,Amazon,348712880,AMZN Mobile LLC,"Umfassend \nSchnell suchen, Produktdetails abr..."
134,亚马逊中国 一站放心购全球,342576766,AMZN Mobile LLC,【关于亚马逊购物】\n全球挚爱的跨境电商购物平台，同步全球品质生活，海外正品直邮， 尽在亚马...


In [471]:
from tqdm import tqdm #可以用这个查看运行进度
import time
top_k_hits =1 
pub_id = list()

ama = [] #空字典，用于存储

#开始循环
for i in tqdm(range(0,len(amazon))):
    for j in range(68466,68467):
        description = amazon.loc[[i]] #对于每一个p
        Split_description = split_des(description).iloc[:,3].tolist() #对于每一个app的description拆句，转化list
        
        abstract = final_abstract_all.loc[[j]] #对于每一个patent
        Split_abstract = split_abstract(abstract).iloc[:,1].tolist() #对于每一个patent的abstract拆句，转化list
        
        Descrip_embed = model.encode(Split_description) ## 768 x description分出的句数 的大小 的矩阵
        Abstract_embed = model.encode(Split_abstract) ## 768 x abstract分出的句数 的大小 的矩阵
        
      
        all_simi = cosine_similarity(Descrip_embed,Abstract_embed)  #所有的similarity， 如拆10句description和
        #3句abstract，这里就是10x3大小的矩阵，里面每个都是similarity，一一对应
        
        #4 rules， 首先把矩阵变成一维的，进行排序
        top_1 = np.sort(all_simi.flatten())[-1:] # 最高的similarity 
        simi5 = np.mean(np.sort(all_simi.flatten())[-5:]) # top5/5
        simi10 = np.mean(np.sort(all_simi.flatten())[-10:]) # top10/10
        simi_all_avg = np.mean(all_simi.flatten()) # 所有similarity取平均
        
        ama.append({'app_id': amazon.iloc[i,0],'this_abstract' : final_abstract_all.iloc[j,1], 'now_simi': all_simi,'simi5': simi5, 'simi10':simi10, 'simi_all_avg':simi_all_avg})
        #append 进去

100%|██████████| 136/136 [04:25<00:00,  1.96s/it]


In [472]:
amzn_company = pd.DataFrame(ama)
amzn_company_5 = amzn_company.sort_values(by="simi5", ascending=False) 
amzn_company_10 = amzn_company.sort_values(by="simi10",ascending=False) 
amzn_company_all = amzn_company.sort_values(by="simi_all_avg", ascending=False) 
amzn_company_5
amzn_company_10
amzn_company_all

amzn_company_5.to_csv('amzn_company_5.csv',encoding='utf-8-sig')
amzn_company_10.to_csv('amzn_company_10.csv',encoding='utf-8-sig')
amzn_company_all.to_csv('amzn_company_all.csv',encoding='utf-8-sig')

,app_id,this_abstract,now_simi,simi5,simi10,simi_all_avg
9,"IMDb: Your guide to movies, TV shows, celebrities","<abstract lang=""en"" load-source=""national-offi...","[[0.05155214, 0.11115698, 0.09222488, 0.069937...",0.445401,0.401382,0.145274
35,AWS Console,"<abstract lang=""en"" load-source=""national-offi...","[[0.31453666, 0.35158083, 0.22171897, 0.082566...",0.443868,0.426579,0.208268
84,Twitch: Live Game Streaming,"<abstract lang=""en"" load-source=""national-offi...","[[0.124796666, 0.073716305, 0.08276653, 0.1635...",0.440369,0.408849,0.191675
14,Twitch: Livestream Multiplayer Games & Esports,"<abstract lang=""en"" load-source=""national-offi...","[[0.21461093, 0.20792079, 0.15614796, 0.154781...",0.435815,0.418373,0.206424
100,AWS Console,"<abstract lang=""en"" load-source=""national-offi...","[[0.29384595, 0.335863, 0.20818509, 0.07210632...",0.435121,0.415192,0.213624
...,...,...,...,...,...,...
37,Amazon Prime Now,"<abstract lang=""en"" load-source=""national-offi...","[[0.052858762, 0.017264552, 0.041270792, 0.061...",0.203287,0.185714,0.065525
134,亚马逊中国 一站放心购全球,"<abstract lang=""en"" load-source=""national-offi...","[[0.046836287, 0.10370969, 0.10092941, 0.05359...",0.192085,0.181823,0.106337
64,Amazon Drive,"<abstract lang=""en"" load-source=""national-offi...","[[-0.023786008, 0.07773203, -0.027221836, -0.0...",0.140865,0.115164,0.037550
25,Amazon English - 英語学習 | 英会話学習,"<abstract lang=""en"" load-source=""national-offi...","[[-0.0013957871, 0.049666457, 0.03489715, -0.0...",0.106541,0.073779,0.057296


,app_id,this_abstract,now_simi,simi5,simi10,simi_all_avg
35,AWS Console,"<abstract lang=""en"" load-source=""national-offi...","[[0.31453666, 0.35158083, 0.22171897, 0.082566...",0.443868,0.426579,0.208268
14,Twitch: Livestream Multiplayer Games & Esports,"<abstract lang=""en"" load-source=""national-offi...","[[0.21461093, 0.20792079, 0.15614796, 0.154781...",0.435815,0.418373,0.206424
100,AWS Console,"<abstract lang=""en"" load-source=""national-offi...","[[0.29384595, 0.335863, 0.20818509, 0.07210632...",0.435121,0.415192,0.213624
29,Wing - Book a Courier Delivery,"<abstract lang=""en"" load-source=""national-offi...","[[0.08128336, 0.09724269, 0.08749078, 0.000983...",0.431211,0.414487,0.233101
84,Twitch: Live Game Streaming,"<abstract lang=""en"" load-source=""national-offi...","[[0.124796666, 0.073716305, 0.08276653, 0.1635...",0.440369,0.408849,0.191675
...,...,...,...,...,...,...
37,Amazon Prime Now,"<abstract lang=""en"" load-source=""national-offi...","[[0.052858762, 0.017264552, 0.041270792, 0.061...",0.203287,0.185714,0.065525
134,亚马逊中国 一站放心购全球,"<abstract lang=""en"" load-source=""national-offi...","[[0.046836287, 0.10370969, 0.10092941, 0.05359...",0.192085,0.181823,0.106337
64,Amazon Drive,"<abstract lang=""en"" load-source=""national-offi...","[[-0.023786008, 0.07773203, -0.027221836, -0.0...",0.140865,0.115164,0.037550
25,Amazon English - 英語学習 | 英会話学習,"<abstract lang=""en"" load-source=""national-offi...","[[-0.0013957871, 0.049666457, 0.03489715, -0.0...",0.106541,0.073779,0.057296


,app_id,this_abstract,now_simi,simi5,simi10,simi_all_avg
24,Transparency -,"<abstract lang=""en"" load-source=""national-offi...","[[0.15824914, 0.25597966, 0.20975384, 0.196016...",0.289856,0.264751,0.250148
118,AWS Global Summits,"<abstract lang=""en"" load-source=""national-offi...","[[0.17870279, 0.20004666, 0.17404246, 0.108727...",0.365747,0.343313,0.247105
29,Wing - Book a Courier Delivery,"<abstract lang=""en"" load-source=""national-offi...","[[0.08128336, 0.09724269, 0.08749078, 0.000983...",0.431211,0.414487,0.233101
100,AWS Console,"<abstract lang=""en"" load-source=""national-offi...","[[0.29384595, 0.335863, 0.20818509, 0.07210632...",0.435121,0.415192,0.213624
7,Neighbors by Ring,"<abstract lang=""en"" load-source=""national-offi...","[[0.128544, 0.12279445, 0.15454537, 0.10106682...",0.378446,0.338273,0.210210
...,...,...,...,...,...,...
87,Amazon Drive,"<abstract lang=""en"" load-source=""national-offi...","[[-0.023786012, 0.07773203, -0.027221845, -0.0...",0.264548,0.217998,0.067213
37,Amazon Prime Now,"<abstract lang=""en"" load-source=""national-offi...","[[0.052858762, 0.017264552, 0.041270792, 0.061...",0.203287,0.185714,0.065525
25,Amazon English - 英語学習 | 英会話学習,"<abstract lang=""en"" load-source=""national-offi...","[[-0.0013957871, 0.049666457, 0.03489715, -0.0...",0.106541,0.073779,0.057296
64,Amazon Drive,"<abstract lang=""en"" load-source=""national-offi...","[[-0.023786008, 0.07773203, -0.027221836, -0.0...",0.140865,0.115164,0.037550


In [473]:
amzn_company.shape
amzn_company['simi5'].describe()
amzn_company['simi10'].describe()
amzn_company['simi_all_avg'].describe()

(136, 6)

count    136.000000
mean       0.319373
std        0.063369
min        0.084784
25%        0.281342
50%        0.325096
75%        0.365674
max        0.445401
Name: simi5, dtype: float64

count    136.000000
mean       0.292049
std        0.062196
min        0.040033
25%        0.255575
50%        0.291062
75%        0.329749
max        0.426579
Name: simi10, dtype: float64

count    136.000000
mean       0.123902
std        0.042301
min        0.024619
25%        0.092970
50%        0.115897
75%        0.146361
max        0.250148
Name: simi_all_avg, dtype: float64

In [298]:
top_k_hits =1 
pub_id = list()

gmail = [] #空字典，用于存储

#开始循环
for i in range(166,167):
    for j in range(0,len(final_abstract)):
        description = final_app.loc[[i]] #对于每一个p
        Split_description = split_des(description).iloc[:,3].tolist() #对于每一个app的description拆句，转化list
        
        abstract = final_abstract.loc[[j]] #对于每一个patent
        Split_abstract = split_abstract(abstract).iloc[:,1].tolist() #对于每一个patent的abstract拆句，转化list
        
        Descrip_embed = model.encode(Split_description) ## 768 x description分出的句数 的大小 的矩阵
        Abstract_embed = model.encode(Split_abstract) ## 768 x abstract分出的句数 的大小 的矩阵
        
      
        all_simi = cosine_similarity(Descrip_embed,Abstract_embed)  #所有的similarity， 如拆10句description和
        #3句abstract，这里就是10x3大小的矩阵，里面每个都是similarity，一一对应
        
        #4 rules， 首先把矩阵变成一维的，进行排序
        top_1 = np.sort(all_simi.flatten())[-1:] # 最高的similarity 
        simi5 = np.mean(np.sort(all_simi.flatten())[-5:]) # top5/5
        simi10 = np.mean(np.sort(all_simi.flatten())[-10:]) # top10/10
        simi_all_avg = np.mean(all_simi.flatten()) # 所有similarity取平均
        
        gmail.append({'app_id': final_app.iloc[i,0],'ab_id' : final_abstract.iloc[j,1], 'now_simi': all_simi,'simi5': simi5, 'simi10':simi10, 'simi_all_avg':simi_all_avg})
        #append 进去

In [299]:
gmail = pd.DataFrame(gmail)

In [300]:
gmail['simi5'].describe()

count    341.000000
mean       0.334644
std        0.055851
min        0.211130
25%        0.302813
50%        0.333086
75%        0.372549
max        0.578873
Name: simi5, dtype: float64

In [301]:
gmail['simi10'].describe()

count    341.000000
mean       0.294714
std        0.051246
min        0.182759
25%        0.264196
50%        0.294663
75%        0.326268
max        0.507684
Name: simi10, dtype: float64

In [302]:
gmail['simi_all_avg'].describe()

count    341.000000
mean       0.120326
std        0.036348
min        0.050108
25%        0.090044
50%        0.114708
75%        0.150592
max        0.218931
Name: simi_all_avg, dtype: float64

In [303]:
final_app.loc[194:195]

,name,app_id,publisher_name,description
194,"Waze - GPS, Maps, Traffic Alerts & Live Naviga...",com.waze,Waze,Always know what’s happening on the road with ...
195,Socratic,org.socratic.android,Socratic,*** Socratic has been acquired by Google! Expe...


In [304]:
top_k_hits =1 
pub_id = list()

waze = [] #空字典，用于存储

#开始循环
for i in range(194,195):
    for j in range(0,len(final_abstract)):
        description = final_app.loc[[i]] #对于每一个p
        Split_description = split_des(description).iloc[:,3].tolist() #对于每一个app的description拆句，转化list
        
        abstract = final_abstract.loc[[j]] #对于每一个patent
        Split_abstract = split_abstract(abstract).iloc[:,1].tolist() #对于每一个patent的abstract拆句，转化list
        
        Descrip_embed = model.encode(Split_description) ## 768 x description分出的句数 的大小 的矩阵
        Abstract_embed = model.encode(Split_abstract) ## 768 x abstract分出的句数 的大小 的矩阵
        
      
        all_simi = cosine_similarity(Descrip_embed,Abstract_embed)  #所有的similarity， 如拆10句description和
        #3句abstract，这里就是10x3大小的矩阵，里面每个都是similarity，一一对应
        
        #4 rules， 首先把矩阵变成一维的，进行排序
        top_1 = np.sort(all_simi.flatten())[-1:] # 最高的similarity 
        simi5 = np.mean(np.sort(all_simi.flatten())[-5:]) # top5/5
        simi10 = np.mean(np.sort(all_simi.flatten())[-10:]) # top10/10
        simi_all_avg = np.mean(all_simi.flatten()) # 所有similarity取平均
        
        waze.append({'app_id': final_app.iloc[i,0],'ab_id' : final_abstract.iloc[j,1], 'now_simi': all_simi,'simi5': simi5, 'simi10':simi10, 'simi_all_avg':simi_all_avg})
        #append 进去

In [305]:
waze = pd.DataFrame(waze)
waze

,app_id,ab_id,now_simi,simi5,simi10,simi_all_avg
0,"Waze - GPS, Maps, Traffic Alerts & Live Naviga...",A system and method for facilitating clusterin...,"[[0.2255611], [0.27018344], [0.3519449], [0.18...",0.300051,0.261941,0.228561
1,"Waze - GPS, Maps, Traffic Alerts & Live Naviga...","Methods, systems, and apparatus for determinin...","[[0.12714659, 0.068410315, 0.0939507, 0.005891...",0.276768,0.241266,0.119549
2,"Waze - GPS, Maps, Traffic Alerts & Live Naviga...",Systems and methods for predictive conversion ...,"[[0.01569863, -0.040125046, -0.0038919384, 0.0...",0.352058,0.302954,0.118853
3,"Waze - GPS, Maps, Traffic Alerts & Live Naviga...","Methods, systems, and apparatus, including com...","[[0.018284572, 0.021100173, 0.04978558, 0.0721...",0.214739,0.197001,0.084477
4,"Waze - GPS, Maps, Traffic Alerts & Live Naviga...","Methods, systems, and apparatus, including com...","[[-0.016901208, -0.0123907635], [0.031036269, ...",0.158861,0.128418,0.057197
...,...,...,...,...,...,...
336,"Waze - GPS, Maps, Traffic Alerts & Live Naviga...",The present disclosure relates to thermal imag...,"[[0.12378474, 0.031818938, 0.17253377, 0.18297...",0.239424,0.224180,0.115122
337,"Waze - GPS, Maps, Traffic Alerts & Live Naviga...","Methods, systems, and apparatus, including com...","[[0.052691557, 0.13883725, 0.034042798, 0.1443...",0.222529,0.213939,0.116572
338,"Waze - GPS, Maps, Traffic Alerts & Live Naviga...","<abstract lang=""en"" load-source=""national-offi...","[[0.060449526, 0.031818938, 0.17253377, 0.1560...",0.220685,0.205565,0.100325
339,"Waze - GPS, Maps, Traffic Alerts & Live Naviga...","<abstract lang=""en"" load-source=""national-offi...","[[0.07419906, 0.26134112, 0.18946785, 0.183034...",0.351005,0.317994,0.166543


In [306]:
waze['simi5'].describe()

count    341.000000
mean       0.274659
std        0.080077
min        0.145619
25%        0.224247
50%        0.261437
75%        0.309467
max        0.528717
Name: simi5, dtype: float64

In [307]:
waze['simi10'].describe()

count    341.000000
mean       0.248837
std        0.078229
min        0.113406
25%        0.197001
50%        0.232439
75%        0.283572
max        0.502086
Name: simi10, dtype: float64

In [308]:
waze['simi_all_avg'].describe()

count    341.000000
mean       0.129486
std        0.053632
min        0.028128
25%        0.091872
50%        0.117069
75%        0.156244
max        0.284960
Name: simi_all_avg, dtype: float64

In [309]:
final_app.loc[8506:8507]

,name,app_id,publisher_name,description
8506,Apple Music,com.apple.android.music,Apple Inc.,Experience music like never before. Apple Musi...
8507,Beats,com.apple.bnd,Apple Inc.,Download the Beats app to unlock features that...


In [310]:
top_k_hits =1 
pub_id = list()

applemusic = [] #空字典，用于存储

#开始循环
for i in range(8506,8507):
    for j in range(0,len(final_abstract)):
        description = final_app.loc[[i]] #对于每一个p
        Split_description = split_des(description).iloc[:,3].tolist() #对于每一个app的description拆句，转化list
        
        abstract = final_abstract.loc[[j]] #对于每一个patent
        Split_abstract = split_abstract(abstract).iloc[:,1].tolist() #对于每一个patent的abstract拆句，转化list
        
        Descrip_embed = model.encode(Split_description) ## 768 x description分出的句数 的大小 的矩阵
        Abstract_embed = model.encode(Split_abstract) ## 768 x abstract分出的句数 的大小 的矩阵
        
      
        all_simi = cosine_similarity(Descrip_embed,Abstract_embed)  #所有的similarity， 如拆10句description和
        #3句abstract，这里就是10x3大小的矩阵，里面每个都是similarity，一一对应
        
        #4 rules， 首先把矩阵变成一维的，进行排序
        top_1 = np.sort(all_simi.flatten())[-1:] # 最高的similarity 
        simi5 = np.mean(np.sort(all_simi.flatten())[-5:]) # top5/5
        simi10 = np.mean(np.sort(all_simi.flatten())[-10:]) # top10/10
        simi_all_avg = np.mean(all_simi.flatten()) # 所有similarity取平均
        
        applemusic.append({'app_id': final_app.iloc[i,0],'ab_id' : final_abstract.iloc[j,1], 'now_simi': all_simi,'simi5': simi5, 'simi10':simi10, 'simi_all_avg':simi_all_avg})
        #append 进去

In [311]:
applemusic = pd.DataFrame(applemusic)
applemusic['simi5'].describe()

count    341.000000
mean       0.279862
std        0.058847
min        0.127926
25%        0.234395
50%        0.276163
75%        0.315832
max        0.435073
Name: simi5, dtype: float64

In [312]:
applemusic['simi10'].describe()

count    341.000000
mean       0.250101
std        0.054272
min        0.108357
25%        0.211329
50%        0.247406
75%        0.287096
max        0.401317
Name: simi10, dtype: float64

In [313]:
applemusic['simi_all_avg'].describe()

count    341.000000
mean       0.092913
std        0.027404
min        0.022446
25%        0.074220
50%        0.091248
75%        0.105701
max        0.166936
Name: simi_all_avg, dtype: float64

In [314]:
final_app.loc[2036:2037]

,name,app_id,publisher_name,description
2036,高德地图,com.autonavi.minimap,高德软件,高德地图---中国专业的手机地图，超过7亿用户正在使用！高德地图拥有全面精准的地点信息，特色...
2037,"UC Cricket - Live Cricket Scores, news & Cricinfo",com.CricChat.intl,UCWeb Singapore Pte. Ltd.,UC Cricket is one of the best cricket app to w...


In [315]:
top_k_hits =1 
pub_id = list()

gaode = [] #空字典，用于存储

#开始循环
for i in range(2036,2037):
    for j in range(0,len(final_abstract)):
        description = final_app.loc[[i]] #对于每一个p
        Split_description = split_des(description).iloc[:,3].tolist() #对于每一个app的description拆句，转化list
        
        abstract = final_abstract.loc[[j]] #对于每一个patent
        Split_abstract = split_abstract(abstract).iloc[:,1].tolist() #对于每一个patent的abstract拆句，转化list
        
        Descrip_embed = model.encode(Split_description) ## 768 x description分出的句数 的大小 的矩阵
        Abstract_embed = model.encode(Split_abstract) ## 768 x abstract分出的句数 的大小 的矩阵
        
      
        all_simi = cosine_similarity(Descrip_embed,Abstract_embed)  #所有的similarity， 如拆10句description和
        #3句abstract，这里就是10x3大小的矩阵，里面每个都是similarity，一一对应
        
        #4 rules， 首先把矩阵变成一维的，进行排序
        top_1 = np.sort(all_simi.flatten())[-1:] # 最高的similarity 
        simi5 = np.mean(np.sort(all_simi.flatten())[-5:]) # top5/5
        simi10 = np.mean(np.sort(all_simi.flatten())[-10:]) # top10/10
        simi_all_avg = np.mean(all_simi.flatten()) # 所有similarity取平均
        
        gaode.append({'app_id': final_app.iloc[i,0],'ab_id' : final_abstract.iloc[j,1], 'now_simi': all_simi,'simi5': simi5, 'simi10':simi10, 'simi_all_avg':simi_all_avg})
        #append 进去

In [316]:
gaode=pd.DataFrame(gaode)
gaode

,app_id,ab_id,now_simi,simi5,simi10,simi_all_avg
0,高德地图,A system and method for facilitating clusterin...,"[[0.35662755], [0.41796464], [0.27095467], [0....",0.441053,0.412392,0.357292
1,高德地图,"Methods, systems, and apparatus for determinin...","[[0.11156683, 0.07853635, 0.16814591, 0.028056...",0.302836,0.270297,0.138502
2,高德地图,Systems and methods for predictive conversion ...,"[[0.14402008, 0.14293717, 0.14894773, 0.157219...",0.374746,0.351356,0.170376
3,高德地图,"Methods, systems, and apparatus, including com...","[[0.14402959, 0.20158097, 0.33468935, 0.230017...",0.399024,0.371354,0.200511
4,高德地图,"Methods, systems, and apparatus, including com...","[[0.19898, 0.1768698], [0.2935456, 0.14936477]...",0.309350,0.271792,0.154433
...,...,...,...,...,...,...
336,高德地图,The present disclosure relates to thermal imag...,"[[0.15622589, 0.010119708, 0.26678973, 0.22267...",0.373265,0.359042,0.202021
337,高德地图,"Methods, systems, and apparatus, including com...","[[0.34557962, 0.2183969, 0.34842256, 0.2997240...",0.467463,0.437184,0.252980
338,高德地图,"<abstract lang=""en"" load-source=""national-offi...","[[0.14710015, 0.010119708, 0.26678973, 0.22748...",0.375192,0.361337,0.193693
339,高德地图,"<abstract lang=""en"" load-source=""national-offi...","[[0.23973826, 0.29665747, 0.2766228, 0.2762713...",0.443061,0.421295,0.268972


In [317]:
gaode['simi5'].describe()

count    341.000000
mean       0.398312
std        0.075898
min        0.254852
25%        0.341721
50%        0.391027
75%        0.442136
max        0.618624
Name: simi5, dtype: float64

In [318]:
gaode['simi10'].describe()

count    341.000000
mean       0.371610
std        0.073366
min        0.228354
25%        0.316473
50%        0.363678
75%        0.412392
max        0.592019
Name: simi10, dtype: float64

In [319]:
gaode['simi_all_avg'].describe()

count    341.000000
mean       0.215004
std        0.060080
min        0.109087
25%        0.165673
50%        0.206788
75%        0.254779
max        0.372951
Name: simi_all_avg, dtype: float64

In [320]:
final_app.loc[100644:100646]

,name,app_id,publisher_name,description
100644,The Wall Street Journal.,364387007,"Dow Jones & Company, Inc., publisher of The Wa...",Stay ahead of the competition with the app tha...
100645,Barron’s - Investing Insights,409882593,"Dow Jones & Company, Inc., publisher of The Wa...","Navigate global markets with Barron’s, the wor..."
100646,WSJ China for iPad,438496614,"Dow Jones & Company, Inc., publisher of The Wa...",Get on-the-go access to news from The Wall Str...


In [321]:
top_k_hits =1 
pub_id = list()

wsj = [] #空字典，用于存储

#开始循环
for i in range(100644,100645):
    for j in range(0,len(final_abstract)):
        description = final_app.loc[[i]] #对于每一个p
        Split_description = split_des(description).iloc[:,3].tolist() #对于每一个app的description拆句，转化list
        
        abstract = final_abstract.loc[[j]] #对于每一个patent
        Split_abstract = split_abstract(abstract).iloc[:,1].tolist() #对于每一个patent的abstract拆句，转化list
        
        Descrip_embed = model.encode(Split_description) ## 768 x description分出的句数 的大小 的矩阵
        Abstract_embed = model.encode(Split_abstract) ## 768 x abstract分出的句数 的大小 的矩阵
        
      
        all_simi = cosine_similarity(Descrip_embed,Abstract_embed)  #所有的similarity， 如拆10句description和
        #3句abstract，这里就是10x3大小的矩阵，里面每个都是similarity，一一对应
        
        #4 rules， 首先把矩阵变成一维的，进行排序
        top_1 = np.sort(all_simi.flatten())[-1:] # 最高的similarity 
        simi5 = np.mean(np.sort(all_simi.flatten())[-5:]) # top5/5
        simi10 = np.mean(np.sort(all_simi.flatten())[-10:]) # top10/10
        simi_all_avg = np.mean(all_simi.flatten()) # 所有similarity取平均
        
        wsj.append({'app_id': final_app.iloc[i,0],'ab_id' : final_abstract.iloc[j,1], 'now_simi': all_simi,'simi5': simi5, 'simi10':simi10, 'simi_all_avg':simi_all_avg})
        #append 进去

In [322]:
wsj=pd.DataFrame(wsj)
wsj

,app_id,ab_id,now_simi,simi5,simi10,simi_all_avg
0,The Wall Street Journal.,A system and method for facilitating clusterin...,"[[0.17002514], [0.23751938], [0.13100809], [0....",0.273267,0.255117,0.156107
1,The Wall Street Journal.,"Methods, systems, and apparatus for determinin...","[[0.08160752, 0.16836272, 0.110066056, 0.14216...",0.332265,0.297874,0.085848
2,The Wall Street Journal.,Systems and methods for predictive conversion ...,"[[0.08895374, 0.10409296, 0.10658648, 0.196233...",0.408205,0.371041,0.134920
3,The Wall Street Journal.,"Methods, systems, and apparatus, including com...","[[0.18196936, 0.11618757, 0.15779504, 0.140570...",0.302521,0.284899,0.095161
4,The Wall Street Journal.,"Methods, systems, and apparatus, including com...","[[0.107276544, 0.056137793], [0.21425639, 0.14...",0.236282,0.215549,0.078215
...,...,...,...,...,...,...
336,The Wall Street Journal.,The present disclosure relates to thermal imag...,"[[0.04606639, 0.116931364, 0.19213039, 0.09847...",0.371295,0.332747,0.092854
337,The Wall Street Journal.,"Methods, systems, and apparatus, including com...","[[0.14077666, 0.17798519, 0.09731643, 0.018027...",0.364243,0.327881,0.106106
338,The Wall Street Journal.,"<abstract lang=""en"" load-source=""national-offi...","[[0.0068674805, 0.116931364, 0.19213039, 0.084...",0.361883,0.334322,0.085980
339,The Wall Street Journal.,"<abstract lang=""en"" load-source=""national-offi...","[[0.085474, 0.24871774, 0.18843944, 0.09347289...",0.364300,0.340221,0.126747


In [323]:
wsj['simi5'].describe()

count    341.000000
mean       0.339780
std        0.063459
min        0.219996
25%        0.293390
50%        0.336603
75%        0.382134
max        0.529587
Name: simi5, dtype: float64

In [324]:
wsj['simi10'].describe()

count    341.000000
mean       0.313313
std        0.058731
min        0.200267
25%        0.269156
50%        0.310510
75%        0.356435
max        0.487575
Name: simi10, dtype: float64

In [325]:
wsj['simi_all_avg'].describe()

count    341.000000
mean       0.111686
std        0.034469
min        0.028265
25%        0.083164
50%        0.103889
75%        0.136899
max        0.203158
Name: simi_all_avg, dtype: float64

In [ ]:
final_app.loc[676:677]

In [326]:
top_k_hits =1 
pub_id = list()

wz = [] #空字典，用于存储

#开始循环
for i in range(676,677):
    for j in range(0,len(final_abstract)):
        description = final_app.loc[[i]] #对于每一个p
        Split_description = split_des(description).iloc[:,3].tolist() #对于每一个app的description拆句，转化list
        
        abstract = final_abstract.loc[[j]] #对于每一个patent
        Split_abstract = split_abstract(abstract).iloc[:,1].tolist() #对于每一个patent的abstract拆句，转化list
        
        Descrip_embed = model.encode(Split_description) ## 768 x description分出的句数 的大小 的矩阵
        Abstract_embed = model.encode(Split_abstract) ## 768 x abstract分出的句数 的大小 的矩阵
        
      
        all_simi = cosine_similarity(Descrip_embed,Abstract_embed)  #所有的similarity， 如拆10句description和
        #3句abstract，这里就是10x3大小的矩阵，里面每个都是similarity，一一对应
        
        #4 rules， 首先把矩阵变成一维的，进行排序
        top_1 = np.sort(all_simi.flatten())[-1:] # 最高的similarity 
        simi5 = np.mean(np.sort(all_simi.flatten())[-5:]) # top5/5
        simi10 = np.mean(np.sort(all_simi.flatten())[-10:]) # top10/10
        simi_all_avg = np.mean(all_simi.flatten()) # 所有similarity取平均
        
        wz.append({'app_id': final_app.iloc[i,0],'ab_id' : final_abstract.iloc[j,1], 'now_simi': all_simi,'simi5': simi5, 'simi10':simi10, 'simi_all_avg':simi_all_avg})
        #append 进去

In [327]:
wz= pd.DataFrame(wz)
wz['simi5'].describe()

count    341.000000
mean       0.268252
std        0.059511
min        0.133835
25%        0.232732
50%        0.267869
75%        0.308033
max        0.445221
Name: simi5, dtype: float64

In [328]:
wz['simi10'].describe()

count    341.000000
mean       0.243756
std        0.057780
min        0.110773
25%        0.203053
50%        0.245168
75%        0.277841
max        0.401624
Name: simi10, dtype: float64

In [329]:
wz['simi_all_avg'].describe()

count    341.000000
mean       0.095301
std        0.033735
min        0.012405
25%        0.072243
50%        0.097193
75%        0.117560
max        0.180574
Name: simi_all_avg, dtype: float64

In [330]:
final_app.loc[31777:31778]

,name,app_id,publisher_name,description
31777,BBC News,364147881,BBC Worldwide,"The BBC News app brings you the latest, breaki..."
31778,BBC News,377382255,BBC Media Applications Technologies Limited,Get the latest breaking news from the BBC and ...


In [282]:
top_k_hits =1 
pub_id = list()

bbc = [] #空字典，用于存储

#开始循环
for i in range(31778,31779):
    for j in range(0,len(final_abstract)):
        description = final_app.loc[[i]] #对于每一个p
                  #这里的iloc要根据df里面description在第几列去更改！！！！对于当前data是3
        Split_description = split_des(description).iloc[:,3].tolist() #对于每一个app的description拆句，转化list

        abstract = final_abstract.loc[[j]] #对于每一个patent
        Split_abstract = split_abstract(abstract).iloc[:,1].tolist() #对于每一个patent的abstract拆句，转化list
        
        
#         print(Split_description)
#         print(Split_abstract)
        
        Descrip_embed = model.encode(Split_description) ## 768 x description分出的句数 的大小 的矩阵
        Abstract_embed = model.encode(Split_abstract) ## 768 x abstract分出的句数 的大小 的矩阵
        

        all_simi = cosine_similarity(Descrip_embed,Abstract_embed)  #所有的similarity， 如拆10句description和
        #3句abstract，这里就是10x3大小的矩阵，里面每个都是similarity，一一对应
        
#         print(np.mean(np.sort(all_simi.flatten())[-5:]))
#         print(np.mean(np.sort(all_simi.flatten())[-10:]))
#         print(np.sort(all_simi.flatten())[-5:])
#         print(np.sort(all_simi.flatten())[-10:])
        #4 rules， 首先把矩阵变成一维的，进行排序
        top_1 = np.sort(all_simi.flatten())[-1:] # 最高的similarity 
        simi5 = np.mean(np.sort(all_simi.flatten())[-5:]) # top5/5
        simi10 = np.mean(np.sort(all_simi.flatten())[-10:]) # top10/10
        simi_all_avg = np.mean(all_simi.flatten()) # 所有similarity取平均
        
        bbc.append({'app_id': final_app.iloc[i,0],'ab_id' : final_abstract.iloc[j,1], 'now_simi': all_simi,'simi5': simi5, 'simi10':simi10, 'simi_all_avg':simi_all_avg})
        #append 进去

In [283]:
bbc= pd.DataFrame(bbc)
bbc

,app_id,ab_id,now_simi,simi5,simi10,simi_all_avg
0,BBC News,A system and method for facilitating clusterin...,"[[0.14282668], [0.21928349], [0.25824475], [0....",0.332905,0.306566,0.214963
1,BBC News,"Methods, systems, and apparatus for determinin...","[[0.05774281, -0.00028095674, 0.053264476, 0.0...",0.368450,0.318894,0.108934
2,BBC News,Systems and methods for predictive conversion ...,"[[0.14314103, 0.057919644, 0.07186325, 0.07408...",0.419601,0.399310,0.170238
3,BBC News,"Methods, systems, and apparatus, including com...","[[0.051100235, 0.053991966, 0.045803957, 0.045...",0.340156,0.322005,0.132899
4,BBC News,"Methods, systems, and apparatus, including com...","[[0.047497712, 0.055497915], [0.10430054, 0.09...",0.309881,0.269556,0.130290
...,...,...,...,...,...,...
336,BBC News,The present disclosure relates to thermal imag...,"[[0.10158082, -0.016885329, 0.12106339, 0.0528...",0.448502,0.405327,0.134284
337,BBC News,"Methods, systems, and apparatus, including com...","[[0.121233135, 0.14138421, 0.10310346, 0.17545...",0.377808,0.353453,0.131851
338,BBC News,"<abstract lang=""en"" load-source=""national-offi...","[[0.09868002, -0.016885329, 0.12106339, 0.0278...",0.451255,0.404661,0.128575
339,BBC News,"<abstract lang=""en"" load-source=""national-offi...","[[0.1031898, 0.124394, 0.17654195, 0.18503514,...",0.496598,0.465056,0.205915


In [284]:
bbc['simi5'].describe()

count    341.000000
mean       0.380903
std        0.060499
min        0.230319
25%        0.332905
50%        0.384123
75%        0.418907
max        0.580545
Name: simi5, dtype: float64

In [285]:
bbc['simi10'].describe()

count    341.000000
mean       0.348339
std        0.057481
min        0.211095
25%        0.306566
50%        0.346303
75%        0.389952
max        0.544957
Name: simi10, dtype: float64

In [286]:
bbc['simi_all_avg'].describe()

count    341.000000
mean       0.155313
std        0.038315
min        0.072267
25%        0.127763
50%        0.144559
75%        0.183840
max        0.271823
Name: simi_all_avg, dtype: float64

In [349]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [350]:
final_app.loc[1439:1440]

,name,app_id,publisher_name,description
1439,FIFA Soccer,com.ea.gp.fifamobile,ELECTRONIC ARTS,Build your Ultimate Team™ or play as your favo...
1440,EA SPORTS™ FIFA 21 Companion,com.ea.gp.fifaultimate,ELECTRONIC ARTS,"This app requires you to have an EA Account, F..."


In [351]:
top_k_hits =1 
pub_id = list()

fifa = [] #空字典，用于存储

#开始循环
for i in range(1439,1440):
    for j in range(0,len(final_abstract)):
        description = final_app.loc[[i]] #对于每一个p
        Split_description = split_des(description).iloc[:,3].tolist() #对于每一个app的description拆句，转化list
        
        abstract = final_abstract.loc[[j]] #对于每一个patent
        Split_abstract = split_abstract(abstract).iloc[:,1].tolist() #对于每一个patent的abstract拆句，转化list
        
        Descrip_embed = model.encode(Split_description) ## 768 x description分出的句数 的大小 的矩阵
        Abstract_embed = model.encode(Split_abstract) ## 768 x abstract分出的句数 的大小 的矩阵
        
      
        all_simi = cosine_similarity(Descrip_embed,Abstract_embed)  #所有的similarity， 如拆10句description和
        #3句abstract，这里就是10x3大小的矩阵，里面每个都是similarity，一一对应
        
        #4 rules， 首先把矩阵变成一维的，进行排序
        top_1 = np.sort(all_simi.flatten())[-1:] # 最高的similarity 
        simi5 = np.mean(np.sort(all_simi.flatten())[-5:]) # top5/5
        simi10 = np.mean(np.sort(all_simi.flatten())[-10:]) # top10/10
        simi_all_avg = np.mean(all_simi.flatten()) # 所有similarity取平均
        
        fifa.append({'app_id': final_app.iloc[i,0],'ab_id' : final_abstract.iloc[j,1], 'now_simi': all_simi,'simi5': simi5, 'simi10':simi10, 'simi_all_avg':simi_all_avg})
        #append 进去
fifa= pd.DataFrame(fifa)
fifa['simi5'].describe()
fifa['simi10'].describe()
fifa['simi_all_avg'].describe()

count    341.000000
mean       0.321188
std        0.053993
min        0.199478
25%        0.285623
50%        0.316090
75%        0.362141
max        0.491264
Name: simi5, dtype: float64

count    341.000000
mean       0.293543
std        0.052100
min        0.181384
25%        0.253811
50%        0.288430
75%        0.331425
max        0.451518
Name: simi10, dtype: float64

count    341.000000
mean       0.102357
std        0.026187
min        0.040669
25%        0.083160
50%        0.100976
75%        0.118136
max        0.164181
Name: simi_all_avg, dtype: float64

In [352]:
final_app.loc[6078:6079]

,name,app_id,publisher_name,description
6078,Amazon,uk.amazon.mShop.android,Amazon,"<div class=""id-app-orig-desc"">Product Descript..."
6079,Audible Suno,com.audible.universal,"Audible, Inc",<b>LISTEN AT HOME OR ON THE GO</b><br>Make ins...


In [353]:
top_k_hits =1 
pub_id = list()

amazon = [] #空字典，用于存储

#开始循环
for i in range(6078,6079):
    for j in range(0,len(final_abstract)):
        description = final_app.loc[[i]] #对于每一个p
        Split_description = split_des(description).iloc[:,3].tolist() #对于每一个app的description拆句，转化list
        
        abstract = final_abstract.loc[[j]] #对于每一个patent
        Split_abstract = split_abstract(abstract).iloc[:,1].tolist() #对于每一个patent的abstract拆句，转化list
        
        Descrip_embed = model.encode(Split_description) ## 768 x description分出的句数 的大小 的矩阵
        Abstract_embed = model.encode(Split_abstract) ## 768 x abstract分出的句数 的大小 的矩阵
        
      
        all_simi = cosine_similarity(Descrip_embed,Abstract_embed)  #所有的similarity， 如拆10句description和
        #3句abstract，这里就是10x3大小的矩阵，里面每个都是similarity，一一对应
        
        #4 rules， 首先把矩阵变成一维的，进行排序
        top_1 = np.sort(all_simi.flatten())[-1:] # 最高的similarity 
        simi5 = np.mean(np.sort(all_simi.flatten())[-5:]) # top5/5
        simi10 = np.mean(np.sort(all_simi.flatten())[-10:]) # top10/10
        simi_all_avg = np.mean(all_simi.flatten()) # 所有similarity取平均
        
        amazon.append({'app_id': final_app.iloc[i,0],'ab_id' : final_abstract.iloc[j,1], 'now_simi': all_simi,'simi5': simi5, 'simi10':simi10, 'simi_all_avg':simi_all_avg})
        #append 进去

amazon= pd.DataFrame(amazon)
amazon['simi5'].describe()
amazon['simi10'].describe()
amazon['simi_all_avg'].describe()

count    341.000000
mean       0.334113
std        0.054479
min        0.195380
25%        0.294654
50%        0.328752
75%        0.370923
max        0.504540
Name: simi5, dtype: float64

count    341.000000
mean       0.300924
std        0.051558
min        0.165788
25%        0.263884
50%        0.298403
75%        0.338292
max        0.436495
Name: simi10, dtype: float64

count    341.000000
mean       0.118390
std        0.034805
min        0.047496
25%        0.094585
50%        0.117138
75%        0.144355
max        0.200090
Name: simi_all_avg, dtype: float64

In [354]:
final_app.loc[40:41]

,name,app_id,publisher_name,description
40,Instagram,com.instagram.android,Instagram,Instagram (from Facebook) allows you to create...
41,Free Basics by Facebook,org.internet,Facebook,"With Free Basics, you can connect to Facebook ..."


In [355]:
top_k_hits =1 
pub_id = list()

Instagram = [] #空字典，用于存储

#开始循环
for i in range(40,41):
    for j in range(0,len(final_abstract)):
        description = final_app.loc[[i]] #对于每一个p
        Split_description = split_des(description).iloc[:,3].tolist() #对于每一个app的description拆句，转化list
        
        abstract = final_abstract.loc[[j]] #对于每一个patent
        Split_abstract = split_abstract(abstract).iloc[:,1].tolist() #对于每一个patent的abstract拆句，转化list
        
        Descrip_embed = model.encode(Split_description) ## 768 x description分出的句数 的大小 的矩阵
        Abstract_embed = model.encode(Split_abstract) ## 768 x abstract分出的句数 的大小 的矩阵
        
      
        all_simi = cosine_similarity(Descrip_embed,Abstract_embed)  #所有的similarity， 如拆10句description和
        #3句abstract，这里就是10x3大小的矩阵，里面每个都是similarity，一一对应
        
        #4 rules， 首先把矩阵变成一维的，进行排序
        top_1 = np.sort(all_simi.flatten())[-1:] # 最高的similarity 
        simi5 = np.mean(np.sort(all_simi.flatten())[-5:]) # top5/5
        simi10 = np.mean(np.sort(all_simi.flatten())[-10:]) # top10/10
        simi_all_avg = np.mean(all_simi.flatten()) # 所有similarity取平均
        
        Instagram.append({'app_id': final_app.iloc[i,0],'ab_id' : final_abstract.iloc[j,1], 'now_simi': all_simi,'simi5': simi5, 'simi10':simi10, 'simi_all_avg':simi_all_avg})
        #append 进去
Instagram= pd.DataFrame(Instagram)
Instagram['simi5'].describe()
Instagram['simi10'].describe()
Instagram['simi_all_avg'].describe()

count    341.000000
mean       0.287347
std        0.067128
min        0.149634
25%        0.241279
50%        0.284824
75%        0.328059
max        0.546351
Name: simi5, dtype: float64

count    341.000000
mean       0.260092
std        0.061209
min        0.133071
25%        0.212185
50%        0.257884
75%        0.302670
max        0.504081
Name: simi10, dtype: float64

count    341.000000
mean       0.129177
std        0.039278
min        0.046650
25%        0.095266
50%        0.126056
75%        0.161004
max        0.240300
Name: simi_all_avg, dtype: float64

In [356]:
final_app.loc[10117:10118]

,name,app_id,publisher_name,description
10117,Twitter,com.twitter.android,"Twitter, Inc.","<b>Join the conversation!</b> <br><br>Retweet,..."
10118,Vine,co.vine.android,"Twitter, Inc.",Dear Vine Community - thank you for all the in...


In [357]:
top_k_hits =1 
pub_id = list()

Twitter = [] #空字典，用于存储

#开始循环
for i in range(10117,10118):
    for j in range(0,len(final_abstract)):
        description = final_app.loc[[i]] #对于每一个p
        Split_description = split_des(description).iloc[:,3].tolist() #对于每一个app的description拆句，转化list
        
        abstract = final_abstract.loc[[j]] #对于每一个patent
        Split_abstract = split_abstract(abstract).iloc[:,1].tolist() #对于每一个patent的abstract拆句，转化list
        
        Descrip_embed = model.encode(Split_description) ## 768 x description分出的句数 的大小 的矩阵
        Abstract_embed = model.encode(Split_abstract) ## 768 x abstract分出的句数 的大小 的矩阵
        
      
        all_simi = cosine_similarity(Descrip_embed,Abstract_embed)  #所有的similarity， 如拆10句description和
        #3句abstract，这里就是10x3大小的矩阵，里面每个都是similarity，一一对应
        
        #4 rules， 首先把矩阵变成一维的，进行排序
        top_1 = np.sort(all_simi.flatten())[-1:] # 最高的similarity 
        simi5 = np.mean(np.sort(all_simi.flatten())[-5:]) # top5/5
        simi10 = np.mean(np.sort(all_simi.flatten())[-10:]) # top10/10
        simi_all_avg = np.mean(all_simi.flatten()) # 所有similarity取平均
        
        Twitter.append({'app_id': final_app.iloc[i,0],'ab_id' : final_abstract.iloc[j,1], 'now_simi': all_simi,'simi5': simi5, 'simi10':simi10, 'simi_all_avg':simi_all_avg})
        #append 进去
Twitter= pd.DataFrame(Twitter)
Twitter['simi5'].describe()
Twitter['simi10'].describe()
Twitter['simi_all_avg'].describe()

count    341.000000
mean       0.299724
std        0.064908
min        0.156462
25%        0.252748
50%        0.302517
75%        0.335024
max        0.524716
Name: simi5, dtype: float64

count    341.000000
mean       0.276039
std        0.061884
min        0.139681
25%        0.235794
50%        0.279532
75%        0.313416
max        0.497281
Name: simi10, dtype: float64

count    341.000000
mean       0.115502
std        0.042688
min        0.030906
25%        0.084605
50%        0.112411
75%        0.147082
max        0.240889
Name: simi_all_avg, dtype: float64

In [358]:
final_app.loc[1962:1963]

,name,app_id,publisher_name,description
1962,钉钉,930368978,"DingTalk (China) Information Technology Co., Ltd.","DingTalk, an innovative teamwork App by Alibab..."
1963,阿里邮箱,923828102,"ALIBABA (CHINA) CO., LTD",-阿里邮箱，阿里巴巴出品。做专业的办公协同工具。\n-中国最早的企业邮箱服务商之一，20年专...


In [359]:
top_k_hits =1 
pub_id = list()

dd = [] #空字典，用于存储

#开始循环
for i in range(1962,1963):
    for j in range(0,len(final_abstract)):
        description = final_app.loc[[i]] #对于每一个p
        Split_description = split_des(description).iloc[:,3].tolist() #对于每一个app的description拆句，转化list
        
        abstract = final_abstract.loc[[j]] #对于每一个patent
        Split_abstract = split_abstract(abstract).iloc[:,1].tolist() #对于每一个patent的abstract拆句，转化list
        
        Descrip_embed = model.encode(Split_description) ## 768 x description分出的句数 的大小 的矩阵
        Abstract_embed = model.encode(Split_abstract) ## 768 x abstract分出的句数 的大小 的矩阵
        
      
        all_simi = cosine_similarity(Descrip_embed,Abstract_embed)  #所有的similarity， 如拆10句description和
        #3句abstract，这里就是10x3大小的矩阵，里面每个都是similarity，一一对应
        
        #4 rules， 首先把矩阵变成一维的，进行排序
        top_1 = np.sort(all_simi.flatten())[-1:] # 最高的similarity 
        simi5 = np.mean(np.sort(all_simi.flatten())[-5:]) # top5/5
        simi10 = np.mean(np.sort(all_simi.flatten())[-10:]) # top10/10
        simi_all_avg = np.mean(all_simi.flatten()) # 所有similarity取平均
        
        dd.append({'app_id': final_app.iloc[i,0],'ab_id' : final_abstract.iloc[j,1], 'now_simi': all_simi,'simi5': simi5, 'simi10':simi10, 'simi_all_avg':simi_all_avg})
        #append 进去
dd= pd.DataFrame(dd)
dd['simi5'].describe()
dd['simi10'].describe()
dd['simi_all_avg'].describe()

count    341.000000
mean       0.297085
std        0.064327
min        0.190500
25%        0.253646
50%        0.292088
75%        0.335655
max        0.541132
Name: simi5, dtype: float64

count    341.000000
mean       0.274035
std        0.061733
min        0.168456
25%        0.232498
50%        0.268267
75%        0.305169
max        0.515952
Name: simi10, dtype: float64

count    341.000000
mean       0.162430
std        0.045708
min        0.079151
25%        0.127772
50%        0.155144
75%        0.194907
max        0.281211
Name: simi_all_avg, dtype: float64

In [360]:
final_app.loc[20123:20124]

,name,app_id,publisher_name,description
20123,爱奇艺,393765873,QIYI,【海量视频 抢先看】\n《在远方》：刘烨马伊琍再现时代风云，展现“中国速度”20年\n《世界...
20124,"爱奇艺动画片-益智早教,儿歌故事,中英双语视频-儿童专属播放器",466392993,QIYI,*号外：这个APP已经全新升级，增加收藏功能，请搜索“爱奇艺动画屋”并下载！\n\n(^_^...


In [361]:
top_k_hits =1 
pub_id = list()

aqy = [] #空字典，用于存储

#开始循环
for i in range(20123,20124):
    for j in range(0,len(final_abstract)):
        description = final_app.loc[[i]] #对于每一个p
        Split_description = split_des(description).iloc[:,3].tolist() #对于每一个app的description拆句，转化list
        
        abstract = final_abstract.loc[[j]] #对于每一个patent
        Split_abstract = split_abstract(abstract).iloc[:,1].tolist() #对于每一个patent的abstract拆句，转化list
        
        Descrip_embed = model.encode(Split_description) ## 768 x description分出的句数 的大小 的矩阵
        Abstract_embed = model.encode(Split_abstract) ## 768 x abstract分出的句数 的大小 的矩阵
        
      
        all_simi = cosine_similarity(Descrip_embed,Abstract_embed)  #所有的similarity， 如拆10句description和
        #3句abstract，这里就是10x3大小的矩阵，里面每个都是similarity，一一对应
        
        #4 rules， 首先把矩阵变成一维的，进行排序
        top_1 = np.sort(all_simi.flatten())[-1:] # 最高的similarity 
        simi5 = np.mean(np.sort(all_simi.flatten())[-5:]) # top5/5
        simi10 = np.mean(np.sort(all_simi.flatten())[-10:]) # top10/10
        simi_all_avg = np.mean(all_simi.flatten()) # 所有similarity取平均
        
        aqy.append({'app_id': final_app.iloc[i,0],'ab_id' : final_abstract.iloc[j,1], 'now_simi': all_simi,'simi5': simi5, 'simi10':simi10, 'simi_all_avg':simi_all_avg})
        #append 进去
aqy= pd.DataFrame(aqy)
aqy['simi5'].describe()
aqy['simi10'].describe()
aqy['simi_all_avg'].describe()

count    341.000000
mean       0.258077
std        0.061026
min        0.146203
25%        0.213034
50%        0.248461
75%        0.297395
max        0.477466
Name: simi5, dtype: float64

count    341.000000
mean       0.230433
std        0.058426
min        0.123056
25%        0.189931
50%        0.224097
75%        0.273555
max        0.427586
Name: simi10, dtype: float64

count    341.000000
mean       0.108813
std        0.034337
min        0.046278
25%        0.078036
50%        0.103214
75%        0.137279
max        0.213287
Name: simi_all_avg, dtype: float64

In [362]:
final_app.loc[9194:9195]

,name,app_id,publisher_name,description
9194,超级计算器 — 你随身的数学好帮手,1130826879,NetEase Youdao,【应用介绍】\n迄今为止功能最强大的手机计算器来了！\n超级计算器是网易有道出品的计算app...
9195,网易新闻-头条新闻视频资讯平台,425349261,"NetEase Media Technology (Beijing) Co., Ltd.",【Precise recommendation】\nThe algorithm based ...


In [363]:
top_k_hits =1 
pub_id = list()

jisuanqi = [] #空字典，用于存储

#开始循环
for i in range(9194,9195):
    for j in range(0,len(final_abstract)):
        description = final_app.loc[[i]] #对于每一个p
        Split_description = split_des(description).iloc[:,3].tolist() #对于每一个app的description拆句，转化list
        
        abstract = final_abstract.loc[[j]] #对于每一个patent
        Split_abstract = split_abstract(abstract).iloc[:,1].tolist() #对于每一个patent的abstract拆句，转化list
        
        Descrip_embed = model.encode(Split_description) ## 768 x description分出的句数 的大小 的矩阵
        Abstract_embed = model.encode(Split_abstract) ## 768 x abstract分出的句数 的大小 的矩阵
        
      
        all_simi = cosine_similarity(Descrip_embed,Abstract_embed)  #所有的similarity， 如拆10句description和
        #3句abstract，这里就是10x3大小的矩阵，里面每个都是similarity，一一对应
        
        #4 rules， 首先把矩阵变成一维的，进行排序
        top_1 = np.sort(all_simi.flatten())[-1:] # 最高的similarity 
        simi5 = np.mean(np.sort(all_simi.flatten())[-5:]) # top5/5
        simi10 = np.mean(np.sort(all_simi.flatten())[-10:]) # top10/10
        simi_all_avg = np.mean(all_simi.flatten()) # 所有similarity取平均
        
        jisuanqi.append({'app_id': final_app.iloc[i,0],'ab_id' : final_abstract.iloc[j,1], 'now_simi': all_simi,'simi5': simi5, 'simi10':simi10, 'simi_all_avg':simi_all_avg})
        #append 进去
jisuanqi= pd.DataFrame(jisuanqi)
jisuanqi['simi5'].describe()
jisuanqi['simi10'].describe()
jisuanqi['simi_all_avg'].describe()

count    341.000000
mean       0.320798
std        0.051117
min        0.192216
25%        0.287988
50%        0.319652
75%        0.355401
max        0.501774
Name: simi5, dtype: float64

count    341.000000
mean       0.292685
std        0.048968
min        0.157836
25%        0.263996
50%        0.289305
75%        0.323680
max        0.466953
Name: simi10, dtype: float64

count    341.000000
mean       0.176668
std        0.026954
min        0.111050
25%        0.157890
50%        0.177026
75%        0.195940
max        0.251436
Name: simi_all_avg, dtype: float64

In [364]:
final_app.loc[41639:41640]

,name,app_id,publisher_name,description
41639,"Yelp: Food, Delivery & Reviews",284910350,Yelp,Yelp has over 199 million reviews of businesse...
41640,Pidaia,1038414846,SMACH,At last an app to discover exhibitions in Pari...


In [365]:
top_k_hits =1 
pub_id = list()

yelp = [] #空字典，用于存储

#开始循环
for i in range(41639,41640):
    for j in range(0,len(final_abstract)):
        description = final_app.loc[[i]] #对于每一个p
        Split_description = split_des(description).iloc[:,3].tolist() #对于每一个app的description拆句，转化list
        
        abstract = final_abstract.loc[[j]] #对于每一个patent
        Split_abstract = split_abstract(abstract).iloc[:,1].tolist() #对于每一个patent的abstract拆句，转化list
        
        Descrip_embed = model.encode(Split_description) ## 768 x description分出的句数 的大小 的矩阵
        Abstract_embed = model.encode(Split_abstract) ## 768 x abstract分出的句数 的大小 的矩阵
        
      
        all_simi = cosine_similarity(Descrip_embed,Abstract_embed)  #所有的similarity， 如拆10句description和
        #3句abstract，这里就是10x3大小的矩阵，里面每个都是similarity，一一对应
        
        #4 rules， 首先把矩阵变成一维的，进行排序
        top_1 = np.sort(all_simi.flatten())[-1:] # 最高的similarity 
        simi5 = np.mean(np.sort(all_simi.flatten())[-5:]) # top5/5
        simi10 = np.mean(np.sort(all_simi.flatten())[-10:]) # top10/10
        simi_all_avg = np.mean(all_simi.flatten()) # 所有similarity取平均
        
        yelp.append({'app_id': final_app.iloc[i,0],'ab_id' : final_abstract.iloc[j,1], 'now_simi': all_simi,'simi5': simi5, 'simi10':simi10, 'simi_all_avg':simi_all_avg})
        #append 进去
yelp= pd.DataFrame(yelp)
yelp['simi5'].describe()
yelp['simi10'].describe()
yelp['simi_all_avg'].describe()

count    341.000000
mean       0.291647
std        0.070450
min        0.154975
25%        0.240681
50%        0.286855
75%        0.336262
max        0.489170
Name: simi5, dtype: float64

count    341.000000
mean       0.263328
std        0.064880
min        0.135446
25%        0.218088
50%        0.261212
75%        0.312592
max        0.449951
Name: simi10, dtype: float64

count    341.000000
mean       0.112072
std        0.037391
min        0.038043
25%        0.083625
50%        0.104397
75%        0.139701
max        0.206530
Name: simi_all_avg, dtype: float64

In [ ]:
df.loc[df[‘column_name’].isin(some_values)]